# Código Trabalho 2 (Regressão)

## Definindo os DataFrames de treinamento e de teste

In [ ]:
import pandas as pd

df_train = pd.read_csv('Dataset/conjunto_de_treinamento.csv')
df_test = pd.read_csv('Dataset/conjunto_de_teste.csv')

df_train.shape

## Pré-processamento dos dados

### Funções usadas para "tratar" os dados: 
- Atribuir valores numéricos às variáveis nominais;
- Remover variáveis que não trazem boas contribuições para o resultado final (olhar a sessão de "análise de dados" ao final do código)
- Preenche espaços em branco (null) nas colunas que precisam

In [ ]:
def DataProcessing1(df):
    # Exclui colunas que não serão utilizadas
    df.drop(columns=['Id'], inplace=True)
    df.drop(columns=['diferenciais'], inplace=True)
    df.drop(columns=['tipo_vendedor'], inplace=True)
    df.drop(columns=['area_extra'], inplace=True)
    df.drop(columns=['estacionamento'], inplace=True)

    # Transforma coluna 'tipo' em numérica
    df['tipo'] = df['tipo'].map({'Apartamento': 1, 'Casa': 2})
    df.fillna(0.5, inplace=True)

    return df

In [ ]:
def DataProcessing2(df):
    # Calcula a média dos valores dos imóveis por bairro
    mean_encoded = df.groupby('bairro')['preco'].mean()
    
    # Cria um dicionário com os bairros originais e suas respectivas versões codificadas (valores médios dos imóveis)
    bairros_config = mean_encoded.to_dict()
    
    # Mapeia os valores médios dos imóveis por bairro para todos os bairros
    df['bairro'] = df['bairro'].map(mean_encoded)

    return df, bairros_config


In [ ]:
def CodificarBairroTeste(df, bairros_config):
    # Codifica a coluna 'bairro' do conjunto de teste de acordo com o dicionário criado por DataProcessing2(df_train)
    df['bairro'] = df['bairro'].map(bairros_config)
    df_test['bairro'].fillna(df_test['bairro'].mean(), inplace=True)

    return df


In [ ]:
# Passa os DataFrames pelas funções de pré-processamento
df_train = DataProcessing1(df_train)
df_test = DataProcessing1(df_test)

df_train, bairros_enc = DataProcessing2(df_train)
df_test = CodificarBairroTeste(df_test, bairros_enc)

### Checando o pré-processamento
Verifica se todas as colunas dos DataFrames de treinamento e teste estão completamente preenchidas, e se o formato dos dados é numérico.

In [ ]:
# Conta o número de valores NaN em cada coluna
contagem_nan = df_test.isnull().sum(), df_train.isnull().sum()
# Exibe a contagem de valores NaN
print(contagem_nan[0], '\n\n', contagem_nan[1])


### Separando os dados de treino e teste

In [ ]:
X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values
X_test = df_test.iloc[:, :].values

X_train.shape, X_test.shape


### Treinando o modelo e salvando o resultado

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
# Cria um DataFrame com a coluna 'Id' e 'preco' contendo os valores de y_pred
df_resultado = pd.DataFrame({'Id': range(len(y_pred)), 'preco': y_pred})

# Salva o DataFrame em um arquivo CSV
df_resultado.to_csv('resultadofinal.csv', index=False)
df_resultado.shape

### Análise dos dados
A função "graficos_knn" gera:
- Um gráfico de dispersão das previsões versus os valores reais de acordo com o valor escolhido para "k";
- "n" gráficos de barra mostrando os níveis de correlação de uma coluna com todas as outras. O loop serve para garantir que seja analisada a correlação entre todas as permutações de colunas de forma relativamente eficiente.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def graficos_knn(X_train, y_train, col_index, k):
    # Ajusta o modelo KNN com o número de vizinhos (k) desejado
    model = KNeighborsRegressor(n_neighbors=k)
    model.fit(X_train, y_train)
    
    # Obtém as previsões do modelo
    y_pred = model.predict(X_train)
    colors = np.arange(len(y_train))

    # Cria um gráfico de dispersão das previsões versus os valores reais, com cores diferentes para cada variável
    plt.scatter(y_train, y_pred, c=colors, cmap='rainbow')  # cmap define a paleta de cores desejada
    plt.xlabel('Valor Real')
    plt.ylabel('Valor Previsto')
    plt.title(f'Gráfico de Dispersão KNN (k={k})')
    plt.colorbar(label='Amostras')
    plt.show()
    
    # Calcula a correlação entre a coluna selecionada e as outras colunas
    col_name = df_train.columns[col_index]
    df = pd.DataFrame(X_train, columns=df_train.columns[:-1])
    df['Valor Real'] = y_train
    correlation = df.corr()[col_name]
    
    # Plota um gráfico de barras da correlação
    plt.figure(figsize=(10, 6))
    sns.barplot(x=correlation.index, y=correlation.values)
    plt.xlabel('Colunas')
    plt.ylabel('Correlação')
    plt.title(f'Correlação da Coluna "{col_name}" com as Outras Colunas')
    plt.xticks(rotation=90)
    plt.show()


In [ ]:
for n in range(18):
    graficos_knn(X_train, y_train, n, k=1)